In [19]:
# try:
#     get_ipython().system('pip install pandas_datareader')
#     get_ipython().system('pip install pandas')
#     get_ipython().system('pip install matplotlib')
#     get_ipython().system('pip install requests')
#     get_ipython().system('pip install sympy')
#     get_ipython().system('pip install scipy')
#     get_ipython().system('pip install numpy')
#     get_ipython().system('pip install beautifulsoup4')
#     get_ipython().system('pip install numpy-financial')
# except Exception:
#     pass

In [20]:
import requests
import numpy_financial as npf
import numpy as np
from bs4 import BeautifulSoup
import threading
import re

In [21]:
tickers = []

with open('tickers.csv') as fh:
    for line in fh:
        row = line.strip()
        tickers.append(row)

In [22]:
def rdpModelReturn(payoutRatio, PE, returnOnEquity):
    return (1 + returnOnEquity*(1-payoutRatio))*(1+payoutRatio/PE)-1

In [23]:
# cookies cannot be int only string
# copy this following code to extract cookies from jitta.com and then manual replace number to string for some key
# document.cookie.split(';').map((c) => c.trim().split('=').map(decodeURIComponent)).reduce((a, b) => {try {a[b[0]] = JSON.parse(b[1]);}catch(e) {a[b[0]] = b[1];}return a;}, {});

cookies = {
    "_gid": "GA1.2.1666171758.1680587936",
    "gclid": "undefined",
    "hunt7586-_zldp": "ss+YrC93NjPqF7vKLbOAPFEzKpujuf37zXoWPDOEmLWmo3KEoDowjfvVgOHufVhm7uRfjfIQJjw=",
    "hunt7586-_zldt": "68aa0a2f-eeef-4564-81a5-65e3099fd9b9-1",
    "ajs_user_id": "60d8645aa921b600176fc31e",
    "JDCID": "dc89cc652973e72ed3b483303a4bce5fa991b812a29997522c7401a8782b6cad.5439c914472955e6ecc4ac72cfa2a8c48dfe5358c8de3df5e4b63278cc6e5439705672157f5bbd49210a4b2cc4dec3851703061b2613e31a45162e61f84510a469b436eb405ba50db920a0f8f433b0620c295848890c",
    "_ga": "GA1.1.467613354.1680587936",
    "_ga_BT0RBD348J": "GS1.1.1680586104.7.1.1680587950.44.0.0",
    "ajs_anonymous_id": "944bd501-bbbc-464a-a2bb-1baaba009b9d",
    "jitta_nu": "2"
}

In [24]:
def convert_row_to_data(row):
    r = row.find_all('span')
    c = map(lambda x: p2f(x.text), r)
    return list(c)[:-2]

def p2f(x):
    cx = x.replace('%', '').replace(',', '')
    return None if cx == '- -' or cx == '?' else float(cx)

def rate_of_change(nums):
    return list(map(lambda i: None if nums[i-1] is None or nums[i] is None else nums[i] / nums[i-1] - 1, range(1, len(nums))))

def mean(nums):
    nums = list(filter(lambda x: x is not None, nums))
    return 0.0 if len(nums) == 0 else sum(nums) / len(nums)

def median(nums):
    nums = sorted(filter(lambda x: x is not None, nums))
    n = len(nums)
    if n == 0:
        return 0.0
    if n % 2 == 1:
        return nums[n // 2]
    else:
        return (nums[n // 2 - 1] + nums[n // 2]) / 2


def clean_data(y):
    n = len(y)
    if n == 0:
        return []
    cleaned_y = [0.0] * n
    cleaned_y[0] = y[0] if y[0] is not None else 0.0
    cleaned_y[-1] = y[-1] if y[-1] is not None else 0.0
    for i in range(1, n-1):
        if y[i] is None:
            cleaned_y[i] = 0.0
        elif abs(y[i] - cleaned_y[i-1]) > abs(y[i] - y[i+1]):
            cleaned_y[i] = y[i+1]
        else:
            cleaned_y[i] = y[i-1]
    return cleaned_y


def linear_regression(y):
    cleaned_y = clean_data(y)
    n = len(cleaned_y)
    x = list(range(n))
    x_mean = sum(x) / n
    y_mean = sum(cleaned_y) / n
    cov = sum([(x[i] - x_mean) * (cleaned_y[i] - y_mean) for i in range(n)])
    var = sum([(x[i] - x_mean) ** 2 for i in range(n)])
    slope = cov / var
    intercept = y_mean - slope * x_mean
    y_pred = [slope * i + intercept for i in x]
    roc = 0 if y_mean == 0 else slope / y_mean
    return x, slope, intercept, y_pred, roc, y_mean

def dcf_irr(price, eps, g, dpr):
    tax_rate = 0.15
    adj_dpr = (1 + dpr * (1 - tax_rate))
    cashflows = [price * -1]
    g1 = g
    g2 = 0.5 * g
    g3 = min(0.10, 0.2 * g)
    g4 = min(0.04, 0.1 * g)
    last_eps = eps
    
    for i in range(0, 40):
        if i < 5:
            last_eps = last_eps * (1 + g1)
        elif i < 10:
            last_eps = last_eps * (1 + g2)
        elif i < 20:
            last_eps = last_eps * (1 + g3)
        else:
            last_eps = last_eps * (1 + g4)

        cashflows.append(last_eps * adj_dpr)
    irr = npf.irr(cashflows)
    return 0.0 if np.isnan(irr) else irr

In [25]:
def calculate(ticker):
    with requests.post('https://www.jitta.com/stock/' + ticker + '/factsheet', cookies=cookies) as r:
        soup = BeautifulSoup(r.content, 'lxml')

        try:
            price = float(re.search(
                r"\.price\.latest\":{\"datetime\":\"(.*)\",\"close\":(.*),\"market_cap\"", r.content.decode('utf-8'))[2])

            row = soup.find_all('div', {'class': 'dAGwCX'})

            if len(row) == 0:
                return None
        except:
            return None

        REV = convert_row_to_data(row[0])
        EPS = convert_row_to_data(row[1])
        ROE = convert_row_to_data(row[20])
        ROA = convert_row_to_data(row[21])
        DE = convert_row_to_data(row[23])
        DPR = convert_row_to_data(row[24])

        eps_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else x, EPS)))
        dpr_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else max(0, min(100, x)), DPR)))

        eps = eps_forecast[3][-1]
        g = eps_forecast[4]
        dpr = dpr_forecast[5] / 100

        irr = (dcf_irr(price, eps, g, dpr))

        print(f"{ticker} {irr:.2} {g:.2} {eps:.2} {dpr:.2}")

        return {
            "TICKER": ticker,
            "EPS": eps,
            "G": g,
            "DPR": dpr,
            "IRR": irr,
        }


In [26]:
f = open('stock-screen.csv', 'w')
f.write('ticker|PRICE|EPS|G|DPR|IRR\n')


def fetch_stock(ticker):
    data = calculate(ticker)

    if data == None:
        return

    f.write(ticker + '|' +
            '{:.4f}'.format(data['IRR']) + '|' +
            '{:.4f}'.format(data['G']) + '|' +
            '{:.4f}'.format(data['EPS']) + '|' +
            '{:.4f}'.format(data['DPR']) + '\n'
            )


In [27]:
threads  = []
for ticker in tickers:
    fetch_stock(ticker)

f.close()

[0.19, 0.16, 0.18, -0.76, -0.53, -0.75, -0.48, -0.18, None]
nasdaq:AACG 0.0 0.23 -0.43 0.0
[None, None, None, None, None, None, None, 0.0, -0.17]
nasdaq:AACI 0.0 0.6 -0.064 0.0
[None, None, None, None, None, 0.0, -0.67, -12.4, -2.69]
nasdaq:AADI 0.0 0.53 -1.9 0.0
[3.93, 11.1, 4.65, 2.61, 3.03, 3.79, -18.4, -3.09, 0.19]
nasdaq:AAL 0.078 -2.4 -6.7 0.062
[0.19, 0.19, 0.11, 0.2, -0.054, -0.04, 0.56, 0.19, None]
nasdaq:AAME 0.077 0.021 0.16 0.059
[0.28, 0.65, 1.76, 3.67, -0.11, -3.31, -2.67, -2.01, -2.38]
nasdaq:AAOI 0.0 1.1 -2.5 0.0
[0.8, 0.84, 1.0, 1.01, 0.8, 1.02, 1.49, 1.09, 1.86]
nasdaq:AAON 0.027 0.095 1.5 0.28


KeyboardInterrupt: 